In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
#import seaborn as sns
import scipy
#from tqdm import tqdm 
from scipy.interpolate import CubicSpline
%matplotlib notebook

In [3]:
t1 = 1.0
t2 = 1.0

def Ek(t, kx,ky):
    (xx,yy)=np.meshgrid(kx,ky)
    E = -2*t*(np.cos(xx) + np.cos(yy))
    return E

def lamb_intra(Ek, U, Del,mu):
    if (mu>=0):
        l = -np.sqrt( np.square( Ek ) + np.square(U*Del) )
        l =  l[-l>mu]
    else:
        print('error negative mu')
    return l

def lamb_inter(Ek, x, Del):
    l = np.sqrt( np.square( Ek ) + np.square(x*Del) )    # x = U - J
    return l
def lamb_CDW(Ek, x, Del):
    l = np.sqrt( np.square( Ek ) + np.square(x*Del/2)) # x = (3*U - 5*J)/2
    return l
def lamb_tot(Ek, x, y, z, Del_m, Del_mm, Del_cdw ):
    l = np.sqrt( np.square( Ek ) + np.square(x*Del_m)  + np.square(y*Del_mm)  + np.square(z*Del_cdw/2) )
    # x = U
    # y = U - 2J
    # z = (3*U - 5*J)/2
    return l

In [6]:
""" How many k points in the BZ """

grid_kx = np.linspace(-np.pi, np.pi, 300, endpoint = False)
grid_ky = np.linspace(-np.pi, np.pi, 300, endpoint = False)
grid_Ek = np.zeros((len(grid_kx), len(grid_ky)))
grid_Ek=Ek(t1,grid_kx,grid_ky)
#(xx,yy)=np.meshgrid(grid_kx,grid_ky)

In [7]:
U_array = np.r_[ -30 : 2 : 1000j ]
mu=0
Del_intra_array = np.zeros(len(U_array))
Energy_intra_array = np.zeros(len(U_array))
guess = 0.5

for l in range(len(U_array)):
    U = U_array[l]
    for m in range(100):
        Del = 0.0
        Del = np.sum( 1.0/lamb_intra(grid_Ek, U, guess,mu) )
        Del = -0.5*Del*U/np.square(len(grid_kx))*guess
        if(abs(Del - guess) < 1.0E-4):
            energy_Del = 0.0
            energy_Del0 = 0.0
            
            energy_Del = - np.sum( lamb_intra(grid_Ek, U, Del) )
            energy_Del0 = - np.sum( lamb_intra(grid_Ek, U, 0) )
                    
            energy_Del = 2*( energy_Del/np.square(len(grid_kx)) - U*Del**2.0 )
            energy_Del0 = 2*( energy_Del0/np.square(len(grid_kx)) )
            if(energy_Del < energy_Del0):
                Del_intra_array[l] = Del
                Energy_intra_array[l] = energy_Del
            else:
                Del_intra_array[l] = 0.0
                Energy_intra_array[l] = energy_Del0
            break
            
        elif(m == 99):
            Del_intra_array[l] = 0.0
            
            energy_Del0 = - np.sum( lamb_intra(grid_Ek, U, 0,mu) )
            energy_Del0 = 2*( energy_Del0/np.square(len(grid_kx)) )
            Energy_intra_array[l] = energy_Del0
            
        guess = guess*0.5 + Del*0.5

TypeError: lamb_intra() missing 1 required positional argument: 'mu'